In [1]:
import pandas as pd
import numpy as np
import networkx as nx

# Gene network:

In [6]:
def read_network(network_file_name):
    network = nx.read_edgelist(network_file_name, delimiter=' ')
    return network

def write_network_edge_list(network, output_file_name):
    nx.write_edgelist(network, output_file_name, data=False)

def leverageCentrality(G):
    nodes = list(G.nodes)
    degrees = G.degree
    leverageCentrality ={}
    for node in nodes:
        if(len(list(G.neighbors(node)))==0):
            leverageCentrality[node] = 0
        else:
            leverage=0
            for n in G.neighbors(node):
                leverage += (degrees[node]-degrees[n]) / (degrees[node]+degrees[n])

            leverageCentrality[node] = round(((1/degrees[node]) * leverage),4)

    return leverageCentrality

def averageNeighborDegree(G):
    nodes = list(G.nodes)
    degrees = G.degree
    avgDegree = st.mean([t[1] for t in degrees])
    averageNeighborDegree ={}
    for node in nodes:
        neighborsDegree=0
        if(len(list(G.neighbors(node)))==0):
            averageNeighborDegree[node] = 0
        else:
            for n in G.neighbors(node):
                neighborsDegree += degrees[n]

            averageNeighborDegree[node] = neighborsDegree/len(list(G.neighbors(node)))

    return averageNeighborDegree

def bridgingCentrality(G):
    nodes = list(G.nodes)
    degrees = G.degree
    bet = nx.betweenness_centrality(G)
    bridging_centrality={}

    for node in nodes:
        if(degrees[node]!=0):
            nodeDegreeInverse = degrees[node]**-1
            neighborsDegreeInverse=0
            for n in G.neighbors(node):
                neighborsDegreeInverse += degrees[n]**-1

            bridging_centrality[node] = (nodeDegreeInverse/neighborsDegreeInverse) * bet[node]
        else:
            bridging_centrality[node] = 0
    return bridging_centrality


'''
Functions to get node measures of a network.
Each function produces a file with the name of gene and its correspondent measure
'''
def get_df_degree_centrality(network, output_file_name):
    centrality = nx.degree_centrality(network)
    degree_df = pd.DataFrame([centrality]).T
    degree_df.columns=["degree"]
    degree_df.index.name = 'gene'
    degree_df.to_csv(output_file_name, sep="\t", index=True)

def get_df_closeness_centrality(network, output_file_name):
    centrality = nx.closeness_centrality(network)
    closeness_df = pd.DataFrame([centrality]).T
    closeness_df.columns=["closeness"]
    closeness_df.index.name = 'gene'
    closeness_df.to_csv(output_file_name, sep="\t", index=True)

def get_df_betweenness_centrality(network, output_file_name):
    centrality = nx.betweenness_centrality(network)
    betweenness_df = pd.DataFrame([centrality]).T
    betweenness_df.columns=["betweenness"]
    betweenness_df.index.name = 'gene'
    betweenness_df.to_csv(output_file_name, sep="\t", index=True)

def get_df_eigenvector_centrality(network, output_file_name):
    centrality = nx.eigenvector_centrality(network, max_iter = 1000)
    eigenvector_df = pd.DataFrame([centrality]).T
    eigenvector_df.columns=["eigenvector"]
    eigenvector_df.index.name = 'gene'
    eigenvector_df.to_csv(output_file_name, sep="\t", index=True)

def get_df_kcore(network, output_file_name):
    kc = nx.core_number(network)
    kc_df = pd.DataFrame([kc]).T
    kc_df.columns=["kcore"]
    kc_df.index.name = 'gene'
    kc_df.to_csv(output_file_name, sep="\t", index=True)

def get_df_clustering_coefficient(network, output_file_name):
    cc = {}
    for gene in network.nodes():
        cc[gene] = nx.clustering(network, gene)
    cc_df = pd.DataFrame([cc]).T
    cc_df.columns=["clusteringcoeff"]
    cc_df.index.name = 'gene'
    cc_df.to_csv(output_file_name, sep="\t", index=True)

def get_df_average_neighbors_centrality(network, output_file_name):
    centrality = averageNeighborDegree(network)
    average_neighbors = pd.DataFrame([centrality]).T
    average_neighbors.columns=["average_neighbors"]
    average_neighbors.index.name = 'gene'
    average_neighbors.to_csv(output_file_name, sep="\t", index=True) 

def get_df_leverage_centrality(network, output_file_name):
    centrality = leverageCentrality(network)
    leverage = pd.DataFrame([centrality]).T
    leverage.columns=["leverage"]
    leverage.index.name = 'gene'
    leverage.to_csv(output_file_name, sep="\t", index=True)

def get_df_information_centrality(network, output_file_name):
    #Only One CC
    network = network.subgraph(max(nx.connected_components(network), key=len))
    centrality = nx.information_centrality(network)
    information_df = pd.DataFrame([centrality]).T
    information_df.columns=["information"]
    information_df.index.name = 'gene'
    information_df.to_csv(output_file_name, sep="\t", index=True) 
    
def get_df_bridging_centrality(network, output_file_name):
    centrality = bridgingCentrality(network)
    bridging = pd.DataFrame([centrality]).T
    bridging.columns=["bridging"]
    bridging.index.name = 'gene'
    bridging.to_csv(output_file_name, sep="\t", index=True)
    

# Example of use:
# gene_network_file_name = "./data/Reactome_HPRD_HuRI_HINT.txt"
# output_file_name = "./data/degree_Reactome_HPRD_HuRI_HINT.tsv"
# network = read_network(gene_network_file_name)
# get_df_degree_centrality(network, output_file_name)

# Mutation data:

In [3]:
def get_mutation_count(maf):
    mutation_count = pd.crosstab(maf.Hugo_Symbol, maf.Variant_Classification)
    return mutation_count

def get_patient_count(maf):
    patient_count = pd.pivot_table(maf, index=["Hugo_Symbol"], values=["Tumor_Sample_Barcode"], aggfunc=len)
    patient_count = patient_count.rename(columns={"Tumor_Sample_Barcode": "nr_patients"})
    return patient_count

def filter_by_variants(maf):
    variants_to_keep = ['Frame_Shift_Del','Frame_Shift_Ins','In_Frame_Del','In_Frame_Ins','Missense_Mutation','Nonsense_Mutation','Nonstop_Mutation','Splice_Site','Translation_Start_Site']
    maf = maf[maf.Variant_Classification.isin(variants_to_keep)]
    return maf

def get_mutations_per_patient(maf):
    mutations_per_patient = {}
    patients = set(maf["Tumor_Sample_Barcode"])
    for patient in patients:
        maf_one_gene = maf[maf.Tumor_Sample_Barcode.isin([patient])]
        mutations_per_patient[patient] = len(maf_one_gene)
    return mutations_per_patient

def remove_hipermutated_patients(maf, mutations_per_patient):
    mutations_per_patient_count = list(mutations_per_patient.values())
    q1 = np.quantile(mutations_per_patient_count, .25)
    q2 = np.quantile(mutations_per_patient_count, .50)
    q3 = np.quantile(mutations_per_patient_count, .75)
    iqr = q3 - q1
    threshold_hm = q3 + 4.5 * iqr
    mutations_per_patient_filtered = dict(filter(lambda elem: elem[1] <= threshold_hm, mutations_per_patient.items()))
    maf = maf[maf.Tumor_Sample_Barcode.isin(list(mutations_per_patient_filtered))]
    return maf

def write_maf(maf, output_file_name):
    maf.to_csv(output_file_name, sep='\t', index=False)

def get_df_mutation_data(input_maf_file_name, output_file_name): 
    maf = pd.read_csv(input_maf_file_name, sep="\t", comment='#', usecols=["Hugo_Symbol", "Tumor_Sample_Barcode", "Variant_Classification"])
    mutation_count = get_mutation_count(maf)
    patient_count = get_patient_count(maf)

    genes = set(maf["Hugo_Symbol"])
    
    features = list(patient_count) + list(mutation_count)
    df = pd.DataFrame(index=genes, columns=features)
    df.index.name = 'gene'
    df = df.fillna(0)
    df.update(mutation_count)
    df.update(patient_count)
    df.to_csv(output_file_name, sep="\t", index=True) 

In [4]:
# Get mutation files from cBioPortal, using API
from bravado.client import SwaggerClient

cbioportal = SwaggerClient.from_url('https://www.cbioportal.org/api/api-docs',
                                config={"validate_requests":False,"validate_responses":False})

for a in dir(cbioportal):
    cbioportal.__setattr__(a.replace(' ', '_').lower(), cbioportal.__getattr__(a))

id_study_list = ["laml", "acc", "blca", "lgg", "brca", "cesc", "chol", "coadread", "dlbc", "esca",
                 "gbm", "hnsc", "kich", "kirc", "kirp", "lihc", "luad", "lusc", "meso", "ov",
                 "paad", "pcpg", "prad", "sarc", "skcm", "stad", "tgct", "thym", "thca", "ucs",
                 "ucec", "uvm"]

for id_study in id_study_list:
    id_study = id_study + "_tcga_pan_can_atlas_2018"

    muts = cbioportal.mutations.getMutationsInMolecularProfileBySampleListIdUsingGET(
        molecularProfileId = id_study +"_mutations", # {study_id}_mutations gives default mutations profile for study 
        sampleListId = id_study + "_all", # {study_id}_all includes all samples
        projection="DETAILED" # include gene info
    ).result()

    maf=[]  
    for mut in muts:
        line=[]
        line.append(mut["gene"]["hugoGeneSymbol"])
        line.append(mut["sampleId"])
        line.append(mut["mutationType"])
        maf.append(line)
        
    maf = pd.DataFrame(maf,columns = ["Hugo_Symbol", "Tumor_Sample_Barcode", "Variant_Classification"])
    num_genes = len(set(maf["Hugo_Symbol"]))
    num_patients = len(set(maf["Tumor_Sample_Barcode"]))
    num_mutations = maf.shape[0]
    print(id_study, num_genes, num_patients, num_mutations)
    write_maf(maf, "./data/MAFs/"+id_study+"_raw.maf")

laml_tcga_pan_can_atlas_2018 4831 200 7284
acc_tcga_pan_can_atlas_2018 5546 91 8108
blca_tcga_pan_can_atlas_2018 17115 410 102469
lgg_tcga_pan_can_atlas_2018 11817 511 28754
brca_tcga_pan_can_atlas_2018 17171 1066 94948
cesc_tcga_pan_can_atlas_2018 15466 291 56952
chol_tcga_pan_can_atlas_2018 1828 36 2125
coadread_tcga_pan_can_atlas_2018 18349 534 209731
dlbc_tcga_pan_can_atlas_2018 3343 41 4940
esca_tcga_pan_can_atlas_2018 11369 182 26786
gbm_tcga_pan_can_atlas_2018 14182 396 47139
hnsc_tcga_pan_can_atlas_2018 16089 515 76774
kich_tcga_pan_can_atlas_2018 1914 65 2170
kirc_tcga_pan_can_atlas_2018 10293 402 20785
kirp_tcga_pan_can_atlas_2018 10372 276 20708
lihc_tcga_pan_can_atlas_2018 13447 366 37918
luad_tcga_pan_can_atlas_2018 17491 566 160266
lusc_tcga_pan_can_atlas_2018 17222 484 129714
meso_tcga_pan_can_atlas_2018 2392 86 2813
ov_tcga_pan_can_atlas_2018 13567 523 39504
paad_tcga_pan_can_atlas_2018 10252 177 21110
pcpg_tcga_pan_can_atlas_2018 1798 178 2092
prad_tcga_pan_can_atlas_2

In [7]:
'''
 - Preprocessing Mutation Data
 - Creating data set with mutation data
'''
id_study_list = ["laml", "acc", "blca", "lgg", "brca", "cesc", "chol", "coadread", "dlbc", "esca",
                 "gbm", "hnsc", "kich", "kirc", "kirp", "lihc", "luad", "lusc", "meso", "ov",
                 "paad", "pcpg", "prad", "sarc", "skcm", "stad", "tgct", "thym", "thca", "ucs",
                 "ucec", "uvm"]

maf_list = []
for id_study in id_study_list:
    id_study_raw = "./data/MAFs/" + id_study + "_tcga_pan_can_atlas_2018_raw.maf"
    id_study = id_study + "_tcga_pan_can_atlas_2018"
    maf = pd.read_csv(id_study_raw, sep='\t')
    maf = filter_by_variants(maf)
    mutations_per_patient = get_mutations_per_patient(maf)
    maf = remove_hipermutated_patients(maf, mutations_per_patient)
    num_genes = len(set(maf["Hugo_Symbol"]))
    num_patients = len(set(maf["Tumor_Sample_Barcode"]))
    num_mutations = maf.shape[0]
    print(id_study, num_genes, num_patients, num_mutations)
    write_maf(maf, "./data/MAFs/"+id_study+".maf")
    maf_list.append(maf)
maf_full = pd.concat(maf_list, ignore_index=False)
write_maf(maf_full, "./data/Mutation_Cell2018.maf")

laml_tcga_pan_can_atlas_2018 2105 190 3035
acc_tcga_pan_can_atlas_2018 2317 83 2755
blca_tcga_pan_can_atlas_2018 16467 404 87745
lgg_tcga_pan_can_atlas_2018 7774 505 14931
brca_tcga_pan_can_atlas_2018 14530 977 50678
cesc_tcga_pan_can_atlas_2018 11836 266 28633
chol_tcga_pan_can_atlas_2018 1152 35 1294
coadread_tcga_pan_can_atlas_2018 13288 449 44411
dlbc_tcga_pan_can_atlas_2018 3234 37 4771
esca_tcga_pan_can_atlas_2018 9940 179 21523
gbm_tcga_pan_can_atlas_2018 9282 373 20101
hnsc_tcga_pan_can_atlas_2018 15106 492 61941
kich_tcga_pan_can_atlas_2018 1276 64 1416
kirc_tcga_pan_can_atlas_2018 9596 353 18492
kirp_tcga_pan_can_atlas_2018 9170 266 16662
lihc_tcga_pan_can_atlas_2018 12284 352 31042
luad_tcga_pan_can_atlas_2018 17184 560 152961
lusc_tcga_pan_can_atlas_2018 16738 464 118326
meso_tcga_pan_can_atlas_2018 1923 81 2202
ov_tcga_pan_can_atlas_2018 12276 406 31946
paad_tcga_pan_can_atlas_2018 4319 173 6332
pcpg_tcga_pan_can_atlas_2018 1558 177 1764
prad_tcga_pan_can_atlas_2018 7602 4

In [8]:
print("MAF of 33 types of cancer - TCGA Cell 2018")
print("Number of genes:", len(set(maf_full["Hugo_Symbol"])))
print("Number of patients:", len(set(maf_full["Tumor_Sample_Barcode"])))
print("Number of mutations:", maf_full.shape[0])

MAF of 33 types of cancer - TCGA Cell 2018
Number of genes: 19184
Number of patients: 9741
Number of mutations: 1228126


In [15]:
# creating data set of mutation data:
input_maf_file_name = "./data/Mutation_Cell2018.maf"
get_df_mutation_data(input_maf_file_name, "./data/Mutation_Cell2018.tsv")

# Driver benchmark:

In [12]:
def get_df_benchmark_genes(output_file_name):
    ncg_driver_file_name = "./benchmarks/NCG6_tsgoncogene_2020-05-01.tsv"
    cosmic_driver_file_name = "./benchmarks/CGC_COSMIC_Census_2020-05-01.csv"
    ncg_false_positives_file_name = "./benchmarks/NCG6_false_positives_2020-05-01.txt"
    
    ncg_driver = pd.read_csv(ncg_driver_file_name, sep='\t', usecols=["symbol"])
    cosmic_driver = pd.read_csv(cosmic_driver_file_name, sep=',', usecols=["Gene Symbol"])
    ncg_false_positives = pd.read_csv(ncg_false_positives_file_name, sep='\t', usecols=["symbol"])

    ncg_driver_genes = list(ncg_driver["symbol"])
    cosmic_driver_genes = list(cosmic_driver["Gene Symbol"])
    benchmark_driver_genes = set(ncg_driver_genes) | set(cosmic_driver_genes)
    benchmark_false_positives_genes = set(ncg_false_positives["symbol"])
    genes = benchmark_driver_genes | benchmark_false_positives_genes
    intersect_genes = benchmark_driver_genes & benchmark_false_positives_genes

    benchmark_df = pd.DataFrame(index=genes, columns=["class"])
    for gene in genes:
        if gene in benchmark_false_positives_genes:
            benchmark_df.at[gene, "class"] = "false_positive"
        elif gene in benchmark_driver_genes:
            benchmark_df.at[gene, "class"] = "driver"
    benchmark_df.index.name = 'gene'
    benchmark_df.to_csv(output_file_name, sep="\t", index=True)
    return benchmark_df

benchmark_df = get_df_benchmark_genes("./data/gene_labels.tsv")
benchmark_df["class"].value_counts()

driver            680
false_positive    250
Name: class, dtype: int64

# Creating dataframe:

In [16]:
'''
Combining mutation data, network data and gene labels
 - Input:
   - Mutation_Cell2018.tsv
   - All networks measures files (extracting from Reactome_HPRD_HuRI_HINT.txt)
   - gene_labels.tsv
 - Output:
  - MutationCell2018_Reactome_HPRD_HuRI_HINT.tsv
'''

def create_data_set(mutation_file_name, network_name):
    files = []
    files.append(mutation_file_name)
    files.append("./data/degree_" + network_name + ".tsv")
    files.append("./data/closeness_" + network_name + ".tsv")
    files.append("./data/betweenness_" + network_name + ".tsv")
    files.append("./data/eigenvector_" + network_name + ".tsv")
    files.append("./data/kcore_" + network_name + ".tsv")
    files.append("./data/clusteringcoeff_" + network_name + ".tsv")
    files.append("./data/leverage_" + network_name + ".tsv")
    files.append("./data/information_" + network_name + ".tsv")
    files.append("./data/bridging_" + network_name + ".tsv")
    files.append("./data/average_neighbors_" + network_name + ".tsv")

    class_file = "./data/gene_labels.tsv"

    df = pd.read_csv(files[0], sep="\t")
    df = df.set_index("gene")
    i = 1
    while i < len(files):
        df_i = pd.read_csv(files[i], sep="\t")
        df_i = df_i.set_index("gene")
        df = pd.merge(df, df_i, on="gene")
        i = i + 1

    df_class = pd.read_csv(class_file, sep="\t")
    df_class = df_class.set_index("gene")    
    df = pd.merge(df, df_class, on="gene", how="left")

    output_file_name = "./data/MutationCell2018_"+ network_name + ".tsv"
    df.to_csv(output_file_name, sep="\t", index=True)
    print(output_file_name)


network_name = "Reactome_HPRD_HuRI_HINT"
mutation_file_name = "./data/Mutation_Cell2018.tsv"
create_data_set(mutation_file_name, network_name)

./data/MutationCell2018_Reactome_HPRD_HuRI_HINT.tsv
